In [1]:
import numpy as np
import healpy as hp
import h5py

In [2]:
gal = h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/kids/shear_catalog_kids1000.hdf5','r')


In [3]:
def rd2tp(ra,dec):
        #convert to radian
        phi     = ra/180.0*np.pi
        tht     = (-dec+90)/180.0*np.pi
        return tht,phi

gtht,gphi = rd2tp(gal['shear/ra'][:],gal['shear/dec'][:])
pix     = hp.ang2pix(4096,gtht,gphi)
cc = np.bincount(pix,minlength=hp.nside2npix(4096))
cc[cc>1]=1

del gtht, gphi,pix, 


In [ ]:
def make_random(mask, ramin, ramax, decmin, decmax, N=(2*10**10), nside=4096, seed=100):
    """
    Create N random points within a given mask.
    """

    #print 'Building the random catalog...'
    #set the seed for reproducibility
    np.random.seed(seed)
    
    #phi = ra
    #theta = pi/2 - dec
    
    ra_rand = (np.random.random(N)* (ramax - ramin))+ramin
    print("ra_rand: ",ra_rand)
    '''
    v = np.random.random(N)
    print(len(v))
    vmin = np.cos((90.0+decmin)/180.*np.pi)
    vmax = np.cos((90.0+decmax)/180.*np.pi)
    v *= (vmax-vmin)
    v += vmin
    dec_rand = np.arccos(v)
    np.rad2deg(dec_rand,dec_rand)
    dec_rand -= 90.0
    print("dec_rand: ", dec_rand)

    
    #Converting degrees into radians
    #theta_rand = (90.0 - dec_rand)*np.pi/180.
    phi_rand = ra_rand*np.pi/180.
    pix_rand = hp.ang2pix(nside, theta_rand, phi_rand, nest=False)
    #print(pix_rand)
    
    goodm, = np.where(mask[pix_rand]==1)
    ra_rand = ra_rand[goodm]
    #dec_rand = dec_rand[goodm]
    #print len(ra_rand)
    dec_rand=0
    return ra_rand, dec_rand

In [ ]:
randra,randec = make_random(cc,np.min(gal['shear/ra'][:]),np.max(gal['shear/ra'][:]),
                            np.min(gal['shear/dec'][:]),np.min(gal['shear/dec'][:]))

In [6]:
N=(6*10**9)
ramin = np.min(gal['shear/ra'][:])
ramax = np.max(gal['shear/ra'][:])
decmin = np.min(gal['shear/dec'][:])
decmax = np.min(gal['shear/dec'][:])

np.random.seed(100)


ra_rand = (np.random.random(N)* (ramax - ramin))+ramin

v = np.random.random(N)
vmin = np.cos((90.0+decmin)/180.*np.pi)
vmax = np.cos((90.0+decmax)/180.*np.pi)
v *= (vmax-vmin)
v += vmin
dec_rand = np.arccos(v)

del v, vmax, vmin

np.rad2deg(dec_rand,dec_rand)
dec_rand -= 90.0


#Converting degrees into radians
#theta_rand = (90.0 - dec_rand)*np.pi/180.
#phi_rand = ra_rand*np.pi/180.
pix_rand = hp.ang2pix(4096, (90.0 - dec_rand)*np.pi/180., ra_rand*np.pi/180., nest=False)
#del theta_rand, phi_rand
#print(pix_rand)

#goodm, = np.where(cc[pix_rand]==1)
#del pix_rand
#ra_rand = ra_rand[goodm]
#dec_rand = dec_rand[goodm]
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/kids/randoms.hdf5','w') as f:
    f['randoms/ra'] = ra_rand[np.where(cc[pix_rand]==1)]
    f['randoms/dec'] = dec_rand[np.where(cc[pix_rand]==1)]

In [7]:
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/kids/randoms.hdf5','r') as f:
    print(len(f['randoms/ra'][:]))

404805330


In [ ]:
# make binned randoms:
# I broke down the percentage of objects per bin in binned shear catalog 
# bin0: 7.8%, bin1: 16.9%, bin2: 28.8%, bin3: 21.7%, bin4: 24.7%
# we match that distribution here for the randoms (approx 20x denser than the shear catalog):

In [5]:
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/kids/randoms.hdf5','r') as f:
    with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/kids/binned_randoms.hdf5','w') as g:
        g['randoms/bin_0/ra'] = f['randoms/ra'][:31574815]
        g['randoms/bin_0/dec'] = f['randoms/dec'][:31574815]
        g['randoms/bin_1/ra'] = f['randoms/ra'][31574815:99986915]
        g['randoms/bin_1/dec'] = f['randoms/dec'][31574815:99986915]
        g['randoms/bin_2/ra'] = f['randoms/ra'][99986915:216570850]
        g['randoms/bin_2/dec'] = f['randoms/dec'][99986915:216570850]
        g['randoms/bin_3/ra'] = f['randoms/ra'][216570850:304413606]
        g['randoms/bin_3/dec'] = f['randoms/dec'][216570850:304413606]
        g['randoms/bin_4/ra'] = f['randoms/ra'][304413606:404400522]
        g['randoms/bin_4/dec'] = f['randoms/dec'][304413606:404400522]
        # i'm about 1% short of the full randoms size but i think this is fine